# BIO 265 Final Project - Lily Kalcec, Pre Lavania, and Nicole Haseley

In [3]:
# 1 Import libraries
# standard libraries
import os
import pandas as pd
import numpy as np 
import scipy
from matplotlib import pyplot as plt

# differential expression
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

# dimentionality reduction and clustering
from sklearn.decomposition import PCA
import sklearn.cluster as cluster
from sklearn.preprocessing import StandardScaler
import umap

from statsmodels.stats.multitest import multipletests

In [4]:
# 2 Load in data (final RNAseq readcounts)
RNAseq_data = pd.read_csv("rawcounts-final.csv")
RNAseq_data.drop("ensemble_id_version", axis=1, inplace=True)

In [5]:
RNAseq_data

,CFB2001,CFB2006,CFB2007,CFB2009,CFB2013,CFB2022,CFB2039,CFB2044,CFB2045,CFB2049,...,CFB2146,CFB2149,CFB2152,CFB2159,CFB2160,CFB2163,CFB2166,CFB2169,CFB2171,CFB2186
0,1,4,30,15,4,7,23,19,15,15,...,24,12,14,12,16,7,20,23,20,23
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,741,260,681,1110,640,754,965,1107,906,1105,...,1000,680,533,799,1441,705,1000,1452,1140,1875
3,881,293,782,1013,581,587,884,770,711,1049,...,882,603,512,560,1240,619,763,1504,1011,1421
4,64,30,71,102,72,89,121,124,83,122,...,122,76,40,78,157,85,116,130,117,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60644,14,13,25,38,12,21,23,34,26,36,...,27,20,12,10,28,27,27,45,35,44
60645,2511,1032,2328,2841,3197,1091,3035,1812,1845,3294,...,1773,1722,728,1649,3436,2090,1497,4886,2635,2789
60646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60647,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


# Processing

In [7]:
# 3 separate the samples into control and test (have ntm disease) groups
control_samples = ["CFB2006", "CFB2007", "CFB2009", "CFB2013", "CFB2022", "CFB2039", "CFB2044", "CFB2058", 
                  "CFB2059", "CFB2074", "CFB2075", "CFB2082", "CFB2087", "CFB2088", "CFB2102", "CFB2108", 
                  "CFB2112", "CFB2127", "CFB2134", "CFB2138", "CFB2140", "CFB2142", "CFB2146", "CFB2149", 
                   "CFB2152", "CFB2160", "CFB2166", "CFB2169", "CFB2171", "CFB2186"] 
ntm_samples = ["CFB2001", "CFB2045", "CFB2049", "CFB2060", "CFB2061", "CFB2071", "CFB2084", "CFB2098",
              "CFB2105", "CFB2128", "CFB2159", "CFB2163"]

In [8]:
# 4
# create a dictionary mapping each sample to its condition
metadata_dict = {sample: "Control" for sample in control_samples}
metadata_dict.update({sample: "NTM" for sample in ntm_samples})

# convert to dataframe
metadata = pd.DataFrame.from_dict(metadata_dict, orient="index", columns=["condition"])

# ensure the index matches the column names of count_data
metadata.index.name = "sample"

In [9]:
# 5
metadata = metadata.loc[RNAseq_data.columns]

In [10]:
# 6
metadata['condition'] = metadata['condition'].astype(str)

In [ ]:
# 7
# initialize PyDESeq2 model
dds = DeseqDataSet(
    counts=RNAseq_data.T,  # PyDESeq2 expects samples as rows, genes as columns
    metadata=metadata,
    design_factors=["condition"],  # Needs to be a list
    ref_level="Control"  # Setting "Control" as the reference group
)

# run DESeq2 analysis
dds.deseq2()

# extract results
res = DeseqStats(dds, contrast=["condition", "Control", "NTM"], alpha=0.05)
res.summary()

# get the results as a DataFrame
DE_results_df = res.results_df
display(DE_results_df.head())

/opt/anaconda3/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/var/folders/km/jx84sy451dv01dcs0m4gcrp80000gn/T/ipykernel_2367/2599395339.py:3: DeprecationWarning: ref_level is deprecated and no longer has any effect. It will beremoved in a future release.
  dds = DeseqDataSet(
/var/folders/km/jx84sy451dv01dcs0m4gcrp80000gn/T/ipykernel_2367/2599395339.py:3: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
  dds = DeseqDataSet(
Fitting size factors...


Using None as control genes, passed at DeseqDataSet initialization


... done in 0.41 seconds.

Fitting dispersions...
... done in 26.00 seconds.

Fitting dispersion trend curve...
... done in 1.23 seconds.

Fitting MAP dispersions...
... done in 20.47 seconds.

Fitting LFCs...
... done in 13.74 seconds.

Calculating cook's distance...
... done in 0.27 seconds.

Replacing 134 outlier genes.

Fitting dispersions...
... done in 0.21 seconds.

Fitting MAP dispersions...
... done in 0.22 seconds.

Fitting LFCs...
... done in 0.20 seconds.

Running Wald tests...


# Raw p-values – applying an arbitrary **p-value cutoff of 0.001 and fold change cutoff of 0.5** for signficantly differentially expressed genes (as authors did)

In [ ]:
# 8
#### Using Raw p-values and fold change from DE_results_df (data frame with results from the PyDESeq2 analysis)

# Set p-value threshold and log2 fold-change threshold
pvalue_cutoff = 0.001
log2fc_cutoff = 0.5  # Absolute log2 fold-change threshold

# Remove NaN p-values and create a new DataFrame copy
DE_results_df = DE_results_df.dropna(subset=['pvalue']).copy()

# Compute -log10(p-values)
DE_results_df.loc[:, 'log_pvalue'] = -np.log10(DE_results_df["pvalue"])

# Flag significant genes based on BOTH p-value and log2 fold-change threshold
DE_results_df.loc[:, 'significant'] = (DE_results_df["pvalue"] <= pvalue_cutoff) & (DE_results_df["log2FoldChange"].abs() >= log2fc_cutoff)

# Identify upregulated and downregulated genes
DE_results_df.loc[:, 'upregulated'] = DE_results_df['significant'] & (DE_results_df['log2FoldChange'] > log2fc_cutoff)
DE_results_df.loc[:, 'downregulated'] = DE_results_df['significant'] & (DE_results_df['log2FoldChange'] < -log2fc_cutoff)

# Convert p-values to -log10 scale for plotting
neg_log_pvals = DE_results_df['log_pvalue']

# Create volcano plot
plt.figure(figsize=(6, 5))

# Plot non-significant genes in gray
plt.scatter(
    DE_results_df["log2FoldChange"][~DE_results_df["significant"]],
    neg_log_pvals[~DE_results_df["significant"]],
    c='gray', alpha=0.5, s=10, label="Not Significant"
)

# Plot upregulated genes in red
plt.scatter(
    DE_results_df["log2FoldChange"][DE_results_df["upregulated"]],
    neg_log_pvals[DE_results_df["upregulated"]],
    c='red', alpha=0.7, s=15, label="Upregulated"
)

# Plot downregulated genes in green
plt.scatter(
    DE_results_df["log2FoldChange"][DE_results_df["downregulated"]],
    neg_log_pvals[DE_results_df["downregulated"]],
    c='limegreen', alpha=0.7, s=15, label="Downregulated"
)

# Add threshold lines
plt.axhline(y=-np.log10(pvalue_cutoff), color='red', linestyle='--', linewidth=1)
plt.axvline(x=log2fc_cutoff, color='red', linestyle='--', linewidth=1)
plt.axvline(x=-log2fc_cutoff, color='red', linestyle='--', linewidth=1)

# Labels and title
plt.xlabel("log2FoldChange")
plt.ylabel("-log10(p-value)")
plt.title("Volcano Plot with p-value & Fold-Change Cutoff")
plt.legend(fontsize=9)
plt.show()

# Print the number of significant genes
significant_genes = DE_results_df[DE_results_df['significant']]
print(f"Number of significant genes (p-value ≤ {pvalue_cutoff}, |log2FC| ≥ {log2fc_cutoff}): {len(significant_genes)}")

# Benjamini-Hochberg corrrected p-values – also applying an arbitrary **p-value cutoff of 0.001 and fold change cutoff of 0.5** for signficantly differentially expressed genes (as authors did)

In [ ]:
# 9
#### With Benjamini-Hochberg (BH) correction - using BH-corrected p-values ("padj" values) from the same DE_results_df analysis

# Set p-value threshold and log2 fold-change threshold
padj_threshold = 0.001
log2fc_threshold = 0.5 # Absolute log2 fold-change threshold

# Remove NaN padj values
DE_results_df = DE_results_df.dropna(subset=['padj'])

# Compute -log10(padj)
DE_results_df['log_padj'] = -np.log10(DE_results_df['padj'])

# Determine significance: Significant if padj < threshold and |log2FoldChange| > threshold
DE_results_df['significant'] = (DE_results_df['padj'] < padj_threshold) & (np.abs(DE_results_df['log2FoldChange']) > log2fc_threshold)

# Create volcano plot
plt.figure(figsize=(6, 5))

# Plot non-significant genes in gray
plt.scatter(
    DE_results_df["log2FoldChange"][~DE_results_df["significant"]],
    DE_results_df['log_padj'][~DE_results_df["significant"]],
    c='gray', alpha=0.5, s=10, label="Not Significant")

# Plot significant upregulated genes in red
plt.scatter(
    DE_results_df["log2FoldChange"][(DE_results_df["significant"]) & (DE_results_df["log2FoldChange"] > 0)],
    DE_results_df['log_padj'][(DE_results_df["significant"]) & (DE_results_df["log2FoldChange"] > 0)],
    c='red', alpha=0.7, s=15, label="Upregulated")

# Plot significant downregulated genes in green
plt.scatter(
    DE_results_df["log2FoldChange"][(DE_results_df["significant"]) & (DE_results_df["log2FoldChange"] < 0)],
    DE_results_df['log_padj'][(DE_results_df["significant"]) & (DE_results_df["log2FoldChange"] < 0)],
    c='green', alpha=0.7, s=15, label="Downregulated")

# Labels and title
plt.xlabel("log2FoldChange")
plt.ylabel("-log10(padj)")
plt.title("Volcano Plot (padj < 0.001, |log2FC| > 0.5)")
plt.legend()
plt.show()